### This notebook computes validation score on part of training set

In [1]:
!ls /kaggle/input/
!pip install kornia --no-index --find-links=file:///kaggle/input/imc2022-dependencies/pip/kornia/ --upgrade 
!pip install kornia_moons --no-index --find-links=file:///kaggle/input/imc2022-dependencies/pip/kornia_moons/ --no-deps  --upgrade 
print('Done!')

image-matching-challenge-2022  imc2022-dependencies  loftrutils
imc-utils		       imutils
Looking in links: file:///kaggle/input/imc2022-dependencies/pip/kornia/
Processing /kaggle/input/imc2022-dependencies/pip/kornia/kornia-0.6.4-py2.py3-none-any.whl
  Attempting uninstall: kornia
    Found existing installation: kornia 0.5.8
    Uninstalling kornia-0.5.8:
      Successfully uninstalled kornia-0.5.8
Looking in links: file:///kaggle/input/imc2022-dependencies/pip/kornia_moons/
Processing /kaggle/input/imc2022-dependencies/pip/kornia_moons/kornia_moons-0.1.9-py3-none-any.whl
Done!


In [2]:
!pip install ../input/loftrutils/einops-0.4.1-py3-none-any.whl

Processing /kaggle/input/loftrutils/einops-0.4.1-py3-none-any.whl


In [3]:
!cp -r ../input/imutils/imutils-0.5.3/ /
!pip install /imutils-0.5.3/

Processing /imutils-0.5.3
  Preparing metadata (setup.py) ... - done
  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25852 sha256=93d4c07be97e31450d40b0b1d8b6d2e9adc44d85ba6cb899fdba6f53ef1b3472
  Stored in directory: /root/.cache/pip/wheels/a2/ec/33/8f9b37997c8257074c2ca9ff92a2ab08914f2571fafa5afb2e
Successfully built imutils


In [4]:
import sys
sys.path.append('../input/loftrutils/LoFTR-master/LoFTR-master/')
sys.path.append('../input/imc-utils/')

In [5]:
import os
from collections import namedtuple
import cv2
import kornia as K
import kornia.feature as KF
from kornia.feature.loftr import LoFTR
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import glob
import random
from tqdm.notebook import tqdm
from PIL import Image
import matplotlib.cm as cm

from kornia_moons.feature import *

# from src.loftr import LoFTR, default_cfg
from src.utils.plotting import make_matching_figure

In [6]:
DEVICE = 'cuda:0'
WEIGHT_PATH = '../input/loftrutils/outdoor_ds.ckpt'
LONGEST_EDGE = 1280

In [7]:
matcher = LoFTR(pretrained=None)
matcher.load_state_dict(torch.load(WEIGHT_PATH)['state_dict'])
matcher = matcher.to(DEVICE)
matcher.eval()
print()

In [8]:
import csv

src = '/kaggle/input/image-matching-challenge-2022/'

test_samples = []
with open(f'{src}/test.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        test_samples += [row]

In [9]:
def plot_images(ims):
    
    fig, axes = plt.subplots(3, 3, figsize=(20,20))
    
    for idx, img in enumerate(ims):
        i = idx % 3 
        j = idx // 3 
        image = Image.open(img)
        image = image.resize((300,300))
        axes[i, j].imshow(image)
        axes[i, j].set_title(img.split('/')[-1])

    plt.subplots_adjust(wspace=0, hspace=.2)
    plt.show()

In [10]:
import imutils
def resize_keep_ratio(img, longest_size=LONGEST_EDGE):
    height, width = img.shape[:2]
    if np.maximum(height, width) <= longest_size: # no need to resize
        return img
    
    if height >= width:
        resized_img = imutils.resize(img, height=longest_size)
    else:
        resized_img = imutils.resize(img, width=longest_size)
    return resized_img

def load_torch_image(fname):
    img = cv2.imread(fname)
    img = resize_keep_ratio(img)
    img = cv2.resize(img, (img.shape[1]//8*8, img.shape[0]//8*8))  # input size should be divisible by 8
    img = K.image_to_tensor(img, False).float() /255.
    img = K.color.bgr_to_rgb(img)
    return img

In [11]:
def match(img_path0, img_path1, matcher, device=DEVICE):
    img0 = load_torch_image(img_path0)
    img1 = load_torch_image(img_path1)
        
    input_dict = {"image0": K.color.rgb_to_grayscale(img0).to(device), 
                  "image1": K.color.rgb_to_grayscale(img1).to(device)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
        
    return mkpts0, mkpts1
        
def get_F_matrix(mkpts0, mkpts1):

    # Make sure we do not trigger an exception here.
    if len(mkpts0) > 8:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)

        assert F.shape == (3, 3), 'Malformed F?'
    else:
        F = np.zeros((3, 3))

    return F

In [12]:
def match_and_draw(img_path0, img_path1, matcher, device=DEVICE, drop_outliers=False):
    
    img0 = load_torch_image(img_path0)
    img1 = load_torch_image(img_path1)
    
    print(img0.shape, img1.shape)
    
    input_dict = {"image0": K.color.rgb_to_grayscale(img0).to(device), 
                  "image1": K.color.rgb_to_grayscale(img1).to(device)}
    
    with torch.no_grad():
        correspondences = matcher(input_dict)
        
    mkpts0 = correspondences['keypoints0'].cpu().numpy()
    mkpts1 = correspondences['keypoints1'].cpu().numpy()
    
    if len(mkpts0) > 8:
        F, inliers = cv2.findFundamentalMat(mkpts0, mkpts1, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)

        assert F.shape == (3, 3), 'Malformed F?'
    else:
        F = np.zeros((3, 3))
            
    if drop_outliers:
        print(len(mkpts0))
        mkpts0 = mkpts0[inliers.reshape(-1) > 0]
        mkpts1 = mkpts1[inliers.reshape(-1) > 0]
        inliers = inliers[inliers > 0]
    
        print(len(mkpts0))
    
    draw_LAF_matches(
        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts0).view(1,-1, 2),
                                    torch.ones(mkpts0.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts0.shape[0]).view(1,-1, 1)),

        KF.laf_from_center_scale_ori(torch.from_numpy(mkpts1).view(1,-1, 2),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1, 1),
                                    torch.ones(mkpts1.shape[0]).view(1,-1, 1)),
        torch.arange(mkpts0.shape[0]).view(-1,1).repeat(1,2),
        K.tensor_to_image(img0),
        K.tensor_to_image(img1),
        inliers,
        draw_dict={'inlier_color': (0.2, 1, 0.2),
                   'tentative_color': None, 
                   'feature_color': (0.2, 0.5, 1), 'vertical': False})
    
    del correspondences, input_dict
    torch.cuda.empty_cache()

In [13]:
def plot_matching(samples, files):
    for i in range(samples.shape[1]):
        path0 = files[samples[0][i]]
        path1 = files[samples[1][i]]
        print(f'Matching: {path0} to {path1}')
        match_and_draw(path0, path1, matcher)
        plt.show()
        
def FlattenMatrix(M, num_digits=8):
    '''Convenience function to write CSV files.'''
    
    return ' '.join([f'{v:.{num_digits}e}' for v in M.flatten()])

In [14]:
# Load per-scene scaling factors.

src = '../input/image-matching-challenge-2022/train'

scaling_dict = {}
with open(f'{src}/scaling_factors.csv') as f:
    reader = csv.reader(f, delimiter=',')
    for i, row in enumerate(reader):
        # Skip header.
        if i == 0:
            continue
        scaling_dict[row[0]] = float(row[1])

print(f'Scaling factors: {scaling_dict}')

Scaling factors: {'british_museum': 2.517, 'brandenburg_gate': 7.38, 'buckingham_palace': 18.75, 'colosseum_exterior': 36.99, 'grand_place_brussels': 10.26, 'lincoln_memorial_statue': 1.85, 'notre_dame_front_facade': 1.36, 'pantheon_exterior': 5.41, 'piazza_san_marco': 7.92, 'sacre_coeur': 20.27, 'sagrada_familia': 4.2, 'st_pauls_cathedral': 7.01, 'st_peters_square': 21.48, 'taj_mahal': 20.76, 'temple_nara_japan': 7.79, 'trevi_fountain': 3.67}


In [15]:
from imc_metric import EvaluateSubmission, ReadCovisibilityData, FlattenMatrix

## Evaluate on training set (sample)

In [16]:
max_num_pairs = 5000

sample_ids = []
fund_matrices = []

for scene in scaling_dict.keys():
    covisibility_dict, F_gt_dict = ReadCovisibilityData(f'{src}/{scene}/pair_covisibility.csv')

    # Let's remove pairs with a covisibility below 0.1. Note that the keys are roughly sorted by difficulty, in increasing order, so let's also shuffle before subsampling.
    # Neither matters for the purposes of this exercise, but let's prevent mistakes down the line.
    pairs = list([key for key, covis in covisibility_dict.items() if covis >= 0.1])
    random.shuffle(pairs)
    n = len(pairs)
    pairs = pairs[:max_num_pairs]
    print(f'Loading covisibility data for "{scene}"... kept {len(pairs)} out of {n} covisible pairs')
    
    for pair in tqdm(pairs):
        image_0_id, image_1_id = pair.split('-')
        img_path0 = f'{src}/{scene}/images/{image_0_id}.jpg'
        img_path1 = f'{src}/{scene}/images/{image_1_id}.jpg'
        mkpts0, mkpts1 = match(img_path0, img_path1, matcher)
        F = get_F_matrix(mkpts0, mkpts1)
        F_str = FlattenMatrix(F)

        sample_ids.append(f'phototourism;{scene};{pair}')
        fund_matrices.append(F_str)
    #     break
    
#     break

Loading covisibility data for "british_museum"... kept 5000 out of 14793 covisible pairs


  0%|          | 0/5000 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /usr/local/src/pytorch/aten/src/ATen/native/BinaryOps.cpp:461.)
  return torch.floor_divide(self, other)


Loading covisibility data for "brandenburg_gate"... kept 5000 out of 50575 covisible pairs


  0%|          | 0/5000 [00:00<?, ?it/s]

...

  0%|          | 0/5000 [00:00<?, ?it/s]

Loading covisibility data for "temple_nara_japan"... kept 5000 out of 22716 covisible pairs


  0%|          | 0/5000 [00:00<?, ?it/s]

Loading covisibility data for "trevi_fountain"... kept 5000 out of 243405 covisible pairs


  0%|          | 0/5000 [00:00<?, ?it/s]

In [17]:
df = pd.DataFrame({'sample_id':sample_ids, 'fundamental_matrix':fund_matrices})
df.to_csv('train_pred.csv', index=False)

In [18]:
thresholds_q = np.linspace(1, 10, 10)
thresholds_t = np.geomspace(0.2, 5, 10)

print('--- Evaluate prediction ---')
maa, maa_per_scene, errors_dict_q, errors_dict_t = EvaluateSubmission('train_pred.csv', scaling_dict, thresholds_q, thresholds_t)
for scene, cur_maa in maa_per_scene.items():
    print(f'Scene "{scene}" ({len(errors_dict_q[scene])} pairs), mAA={cur_maa:.05f}')
print()
print(f'Full dataset: mAA={maa:.05f}')
print()

--- Evaluate prediction ---


100%|██████████| 75307/75307 [00:31<00:00, 2396.67it/s]


Scene "british_museum" (5000 pairs), mAA=0.63222
Scene "brandenburg_gate" (5000 pairs), mAA=0.57720
Scene "buckingham_palace" (5000 pairs), mAA=0.31986
Scene "colosseum_exterior" (5000 pairs), mAA=0.25106
Scene "grand_place_brussels" (5000 pairs), mAA=0.31796
Scene "lincoln_memorial_statue" (5000 pairs), mAA=0.79712
Scene "notre_dame_front_facade" (5000 pairs), mAA=0.84306
Scene "pantheon_exterior" (5000 pairs), mAA=0.61048
Scene "piazza_san_marco" (1475 pairs), mAA=0.23132
Scene "sacre_coeur" (5000 pairs), mAA=0.77364
Scene "sagrada_familia" (3832 pairs), mAA=0.70180
Scene "st_pauls_cathedral" (5000 pairs), mAA=0.52948
Scene "st_peters_square" (5000 pairs), mAA=0.27802
Scene "taj_mahal" (5000 pairs), mAA=0.66838
Scene "temple_nara_japan" (5000 pairs), mAA=0.49652
Scene "trevi_fountain" (5000 pairs), mAA=0.67466

Full dataset: mAA=0.54392

